# 5.1 数据透视表

In [1]:
import pandas as pd
import os
import warnings
from common_util.openDataDir import getFullPath

#忽略警告
warnings.filterwarnings('ignore')
#当数值过大时，Jupyter会默认显示科学计数法，这行代码设置不显示科学计数法，保留显示两位小数
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
filePath = getFullPath('第5章 Pandas两大进阶用法', '第5章 数据透视表案例数据.xlsx')
data = pd.read_excel(filePath)
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1944 entries, 0 to 1943
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_id     1944 non-null   object 
 1   pay_month   1944 non-null   object 
 2   province    1944 non-null   object 
 3   city        1944 non-null   object 
 4   product_id  1944 non-null   object 
 5   payment     1944 non-null   float64
 6   ct          1944 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 106.4+ KB
None


,user_id,pay_month,province,city,product_id,payment,ct
0,u000027,2023年1月,浙江省,金华市,PD00054,10.01,1
1,u000027,2023年1月,浙江省,温州市,PD00008,89.79,1
2,u000056,2023年1月,浙江省,绍兴市,PD00176,211.08,1
3,u000209,2023年1月,江苏省,南京市,PD00476,48.92,2
4,u000231,2023年1月,江苏省,徐州市,PD00237,14.90,1


### 1.计算各省份的总销售金额（payment）和总购买商品件数（ct）

In [3]:
# data：传入要进行透视的原始数据
# index：行分组字段，用什么字段作为索引
# columns：列分组字段
# values：值字段，传入哪个字段的值参与计算
# aggfunc：用什么方式计算，默认用均值
t1 = pd.pivot_table(data, index='province', values=['payment', 'ct'], aggfunc='sum')

t1.sort_values('payment', ascending=False).head()

,ct,payment
province,,
上海市,233,85436.48
浙江省,395,59970.63
福建省,155,24312.67
山东省,130,19757.25
安徽省,200,18359.88


### 2.计算各省、市平均销售金额

In [4]:
t2 = pd.pivot_table(data, index=['province', 'city'], values='payment', aggfunc='mean')

t2.head(4)

payment
province city         
上海市      上海市    540.74
云南省      昆明市    194.78
         普洱市     69.00
         曲靖市     72.27

### 3.计算各省、市不同月份的金额

In [6]:
t2 = pd.pivot_table(data, index=['province', 'city'], columns=['pay_month'], values='payment', aggfunc='sum',
                    fill_value=0)

t2.head(4)

pay_month      2023年1月  2023年2月  2023年3月  2023年4月  2023年5月  2023年6月
province city                                                      
上海市      上海市  20372.08 12855.85 12309.89 17910.81 12421.65  9566.20
云南省      昆明市    415.00   239.00     0.00   125.11     0.00     0.00
         普洱市      0.00     0.00     0.00    69.00     0.00     0.00
         曲靖市    161.50   214.70   195.10   168.00   128.00     0.00

# 5.2 强大又灵活的apply

## 5.2.2 apply计算最好、最差成绩

### 1.背景和思路

背景：我们拿到了一份4位同学三次模拟考试的成绩，想知道每位同学历次模拟中最好成绩和最差成绩分别是多少。

思路：最好和最差，分别对应着max与min，我们先按姓名分组，再用apply函数返回对应的最大和最小值，最后将结果合并。

In [8]:
filePath = getFullPath('第5章 Pandas两大进阶用法', '第5章 apply案例数据.xlsx')
score = pd.read_excel(filePath, sheet_name='成绩表')
score.head(6)

,姓名,科目,综合成绩
0,李华,一模,651
1,李华,二模,579
2,李华,三模,580
3,王雷,一模,475
4,王雷,二模,455
5,王雷,三模,432


#### 每位同学最高成绩

In [11]:
max_score = score.groupby('姓名')['综合成绩'].apply(max).reset_index()
max_score

,姓名,综合成绩
0,张建国,691
1,李华,651
2,李子明,577
3,王雷,475


In [13]:
# 使用上一章的agg 也可以实现同样功能
max_score = score.groupby('姓名')['综合成绩'].agg(['max']).reset_index()
max_score

,姓名,max
0,张建国,691
1,李华,651
2,李子明,577
3,王雷,475


In [12]:
# 最低成绩
min_score = score.groupby('姓名')['综合成绩'].apply(min).reset_index()
min_score

,姓名,综合成绩
0,张建国,553
1,李华,579
2,李子明,490
3,王雷,432


#### 按姓名合并最好和最差成绩

In [14]:
score_comb = pd.merge(max_score, min_score, left_on='姓名', right_on='姓名', how='inner')
score_comb.columns = ['姓名', '最好成绩', '最差成绩']
score_comb

,姓名,最好成绩,最差成绩
0,张建国,691,553
1,李华,651,579
2,李子明,577,490
3,王雷,475,432


In [27]:
# 这个例子使用agg貌似更简单、更合适
score_comb = score.groupby('姓名')['综合成绩'].agg(['max', 'min']).reset_index()
score_comb.columns = ['姓名', '最好成绩', '最差成绩']
score_comb

,姓名,最好成绩,最差成绩
0,张建国,691,553
1,李华,651,579
2,李子明,577,490
3,王雷,475,432


## 5.2.3 筛选每个分组下的第三名

### 1.背景和思路
每个省份销售排名第3的都是哪些城市，以及它们的销售额情况

In [17]:
order = pd.read_excel(filePath, sheet_name='省市销售数据')
print(order.info())
order.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   省份      210 non-null    object
 1   城市      210 non-null    object
 2   近1月销售额  210 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 5.1+ KB
None


,省份,城市,近1月销售额
0,重庆,重庆市,255343
1,浙江省,金华市,302624
2,浙江省,台州市,147853
3,浙江省,舟山市,136547
4,浙江省,杭州市,109073


#### 筛选每个省份销售排名第3的城市，要先进行排序

In [18]:
order_rank = order.sort_values(['省份', '近1月销售额'], ascending=False)
order_rank.head()

,省份,城市,近1月销售额
37,陕西省,西安市,450490
38,陕西省,延安市,120161
39,陕西省,安康市,60456
40,陕西省,汉中市,59391
41,陕西省,咸阳市,47411


#### 定义一个函数，用于筛选排名第三的值

In [24]:
# 入参是一个DataFrame
def get_third(x):
    #如果分组长度小于或等于1，意味着该省份为直辖市
    if len(x) <= 1:
        #返回第0行的所有值，即直辖市本身的数据
        return x.iloc[0, :]

    #针对非直辖市
    else:
        #直接返回第3行（排名第3，索引是2）所有值（城市，近1月销售额）
        return x.iloc[2, :]

#### 将函数和apply结合

In [26]:
order_rank.groupby('省份')[['城市', '近1月销售额']].apply(get_third)

,城市,近1月销售额
省份,,
上海,上海市,139261
云南省,昆明市,203210
内蒙古自治区,兴安盟,258106
北京,北京市,154682
四川省,达州市,427285
天津,天津市,510720
安徽省,合肥市,179518
山东省,菏泽市,187375
山西省,临汾市,186931
